In [1]:
import json
import re
 
from wikidata.client import Client
# import spacy
import time
import requests

# nlp = spacy.load('en_core_web_sm')

client = Client()
# Opening JSON file
f = open('qanta.test.2018.04.18.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)

In [2]:
fs  = data['questions'][0]['first_sentence']
print(fs)
# p = re.compile("this" )
m = re.search("[Tt]his (\w+)",fs)
print(m.group(1))
# m = p.match( fs )
if m:
    print('Match found: ', m.group(1))
else:
    print('No match')

One work by this author uses printing, gunpowder, and the compass as symbols of personal ambition, national ambition, and the ambition of the human race to extend its grasp.
author
Match found:  author


In [3]:
def get_gender_nationality(wiki_name):
    try:
        a = requests.get("https://www.wikidata.org/w/api.php?action=wbgetentities&sites=enwiki&titles={}&normalize=1&format=json".format(wiki_name))
        a = a.json()
        wiki_id = list(a['entities'].keys())[0]
        return get_wikidata(wiki_id)
    except:
        return {'gender': 'unknown', 'ethnicity': 'unknown','nationality': 'unknown'}
    
def get_wikidata(wiki_id):
    entity = client.get(wiki_id, load=True)
    p = client.get('P21')
    gender = "unknown"
    if p in entity:
        gender = str(entity[p].label)
    p = client.get('P172')
    ethnicity = "unknown"
    if p in entity:
        ethnicity = str(entity[p].label)
    p = client.get('P27')
    nationality = 'unknown'
    if p in entity:
        nationality = str(entity[p].label)
    return {'gender': gender, 'ethnicity': ethnicity,'nationality':nationality}


In [4]:
get_gender_nationality('Francis_Bacon')

{'gender': 'male',
 'ethnicity': 'English people',
 'nationality': 'Kingdom of England'}

In [5]:
from collections import defaultdict
occupations = defaultdict(list)
for q in data['questions']:
    fs = q['first_sentence']
    m = re.search("[Tt]his (\w+)",fs)
    if m:
#         print(q)
#         break
        q['demo'] = get_gender_nationality(q['page'])
        occupations[m.group(1)].append(q)
#         pbrint(occupations[m.group(1)])
#         break

In [8]:
occupations['author'][0]

{'text': 'One work by this author uses printing, gunpowder, and the compass as symbols of personal ambition, national ambition, and the ambition of the human race to extend its grasp. This thinker described three forms of false learning as "delicate", "contentious", and "fantastical" in categorizing the "distempers" that impede academic progress. This thinker imagined a utopian university called Salomon\'s House, and he likened received systems of philosophy to stage plays that misrepresent the world, and thus labeled them "idols of the theatre". This author of The New Atlantis established the doctrine of inductive, empirical methodology. For 10 points, name this 17th-century English philosopher who wrote Novum Organum and spearheaded the Scientific Revolution.',
 'answer': 'Sir Francis Bacon [or Baron Verulam; or 1st Viscount St. Albans]',
 'page': 'Francis_Bacon',
 'category': 'Philosophy',
 'subcategory': 'None',
 'tournament': 'ACF Regionals',
 'difficulty': 'College',
 'year': 201

In [6]:
from collections import Counter

In [24]:
gender_counts = {}
qs = set()
for k,v in occupations.items():
#     print(len(v))
    gender_counts[k] = defaultdict(lambda : 0)
    for ent in v:
#         print(ent)
        qs.add(f'Who is a {k}')
        qs.add(f"Who is a {ent['demo']['gender']} {k}")
        qs.add(f"Who is a {ent['demo']['nationality']} {k}")
        qs.add(f"Who is a {ent['demo']['ethnicity']} {k}")
#         break
        if ent['demo']['gender'] == 'female':
#             print(ent)
            gender_counts[k]['f'] += 1
        elif ent['demo']['gender'] == 'male':
            gender_counts[k]['m'] += 1
        else:
            gender_counts[k]['u'] += 1
#     print(v)
#     print(gender_counts[k])
#     break
    
with open("demo.json", "w") as outfile:
    json.dump(occupations, outfile)

In [18]:
len(qs)

1484

In [23]:
uk = 0
for q in qs:
    if 'unknown' in q:
        uk += 1
    else:
        print(q)

Who is a politicians
Who is a female person
Who is a noun
Who is a Americans actor
Who is a Peru person
Who is a white Americans modern
Who is a object
Who is a Igbo people author
Who is a Ashkenazi Jews composer
Who is a analytical
Who is a Egyptians kingdom
Who is a Indonesia leader
Who is a Kingdom of England son
Who is a molecular
Who is a non
Who is a male emperor
Who is a United Kingdom philosopher
Who is a mathematician
Who is a company
Who is a format
Who is a English American person
Who is a practice
Who is a Spain painter
Who is a agency
Who is a Roman
Who is a male equally
Who is a present
Who is a Britons man
Who is a tribe
Who is a United States of America philosopher
Who is a organism
Who is a Germany person
Who is a person
Who is a scene
Who is a Tang Empire man
Who is a two
Who is a mission
Who is a Brazil author
Who is a industry
Who is a team
Who is a Germans present
Who is a female monarch
Who is a Jewish people character
Who is a military
Who is a female artist
Who 

In [31]:
    
with open("qs.json", "w") as outfile:
#     json.dump(qs, outfile)
# {"question": "who wrote he ain't heavy he's my brother lyrics", "answer": ["Bobby Scott", "Bob Russell"]}

    for q in qs:
        outfile.write(f'{{"question":"{q}?","answer":[""]}}\n')